In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter
import platform
from matplotlib import font_manager, rc
if platform.system() == 'Darwin': # macOS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

In [21]:
file_path='./data.xlsx'
sheet_name='Sheet0'

try:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    print("✅ 데이터 로드 성공!")
    
    # 이 셀에서는 df.head()를 넣지 않습니다.
    # 다음 셀에서 확인할 것입니다.
    
except FileNotFoundError:
    print(f"오류: 파일 경로를 찾을 수 없습니다. 파일 이름과 위치({file_path})를 확인하세요.")
except ValueError:
    print(f"오류: 시트 이름을 찾을 수 없습니다. 시트 이름({sheet_name})을 확인하세요.")


# 새로운 셀에 이 코드만 입력하고 실행하세요.
print("--- 데이터프레임 상위 5개 행 ---")
df.head()

✅ 데이터 로드 성공!
--- 데이터프레임 상위 5개 행 ---


/Users/pham/PhamPham2S/Book_Borrowing_Analysis/venv/lib/python3.14/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,번호,도서관,서명,저자,발행자,발행년,ISBN,등록번호,청구기호,대출일,반납일
0,1,화성동탄중앙,매직필 : 기적의 비만 치료제와 살찌지 않는 인간의 탄생,요한 하리 지음 ; 이지연 옮김,어크로스,2025,9791167741905,KTE000143977,513.398-하239ㅁ,2025/11/17,2025/11/30
1,2,봉담,복지의 원리 : 대한민국 복지를 한눈에 꿰뚫는 10가지 이야기,양재진 지음,한겨레,2020,9791160403718,GVE000063054,338.1-양72ㅂ,2025/11/08,2025/11/30
2,3,화성동탄중앙,모든 것이 전쟁이다: 전 세계를 집어삼킨 아마존의 단 하나의 원칙,다나 마티올리 지음 ; 이영래 옮김,21세기북스,2024,9791171179237,KTE000149049,325.1-마839ㅁ,2025/10/27,2025/11/05
3,4,화성동탄중앙,"마음의 여섯 얼굴 : 우울, 불안, 분노, 중독, 광기, 그리고 사랑에 대하여",김건종 지음,에이도스,2019,9791185415338,KTE000113719,181.7-김13ㅁ,2025/10/27,2025/11/05
4,5,병점,기울어진 평등 : 부와 권력은 왜 불평등을 허락하는가,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김",와이즈베리,2025,9791173475535,DBE000094796,332.6-피823ㄱ,2025/10/04,2025/10/19


In [22]:
# 중복 확인 기준 컬럼 지정
# 컬럼 '서명'을 기준으로 중복을 확인합니다.
COLUMN_NAME_FOR_DUPLICATE = '서명'

# 제거 전 행의 개수 확인
print(f"제거 전 행의 개수: {len(df)}개")

# 2. 중복 행 제거 (첫 번째 행만 남기고 삭제)
df_cleaned = df.drop_duplicates(subset=[COLUMN_NAME_FOR_DUPLICATE], keep='first')

# 3. 제거 후 행의 개수 확인
print(f"제거 후 행의 개수: {len(df_cleaned)}개")
print(f"제거된 중복 행 개수: {len(df) - len(df_cleaned)}개")

# 4. 데이터프레임 변수를 df_cleaned로 업데이트
df = df_cleaned
print("\n중복 행 제거가 완료되었으며, df 변수에 저장되었습니다.")

# 새로운 셀에 이 코드만 입력하고 실행하세요.
print("--- 중복 제거 후 데이터프레임 상위 5개 행 ---")
df.head()

제거 전 행의 개수: 807개
제거 후 행의 개수: 708개
제거된 중복 행 개수: 99개

중복 행 제거가 완료되었으며, df 변수에 저장되었습니다.
--- 중복 제거 후 데이터프레임 상위 5개 행 ---


,번호,도서관,서명,저자,발행자,발행년,ISBN,등록번호,청구기호,대출일,반납일
0,1,화성동탄중앙,매직필 : 기적의 비만 치료제와 살찌지 않는 인간의 탄생,요한 하리 지음 ; 이지연 옮김,어크로스,2025,9791167741905,KTE000143977,513.398-하239ㅁ,2025/11/17,2025/11/30
1,2,봉담,복지의 원리 : 대한민국 복지를 한눈에 꿰뚫는 10가지 이야기,양재진 지음,한겨레,2020,9791160403718,GVE000063054,338.1-양72ㅂ,2025/11/08,2025/11/30
2,3,화성동탄중앙,모든 것이 전쟁이다: 전 세계를 집어삼킨 아마존의 단 하나의 원칙,다나 마티올리 지음 ; 이영래 옮김,21세기북스,2024,9791171179237,KTE000149049,325.1-마839ㅁ,2025/10/27,2025/11/05
3,4,화성동탄중앙,"마음의 여섯 얼굴 : 우울, 불안, 분노, 중독, 광기, 그리고 사랑에 대하여",김건종 지음,에이도스,2019,9791185415338,KTE000113719,181.7-김13ㅁ,2025/10/27,2025/11/05
4,5,병점,기울어진 평등 : 부와 권력은 왜 불평등을 허락하는가,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김",와이즈베리,2025,9791173475535,DBE000094796,332.6-피823ㄱ,2025/10/04,2025/10/19


In [23]:
# 1. 제거할 열(Column)의 이름을 리스트로 정의합니다.
columns_to_drop = ['번호', 'ISBN', '등록번호', '반납일']

# 2. .drop() 메서드를 사용하여 열을 제거하고 df에 바로 적용합니다.
# axis=1: 열(Column) 제거
# inplace=True: 원본 df 수정
df.drop(columns=columns_to_drop, axis=1, inplace=True)

print("✅ 지정된 열(번호, ISBN, 등록번호, 반납일) 제거 완료!")

# 새로운 셀에 다음 코드를 입력하고 실행하세요.

print("--- 열 제거 후 데이터 구조 확인 (df.info()) ---")
df.info()

print("\n--- 열 제거 후 데이터프레임 상위 5개 행 (df.head()) ---")
# df.head()를 셀의 마지막 줄에 단독으로 두어 깔끔한 표로 출력합니다.
df.head()

✅ 지정된 열(번호, ISBN, 등록번호, 반납일) 제거 완료!
--- 열 제거 후 데이터 구조 확인 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
Index: 708 entries, 0 to 806
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   도서관     708 non-null    object
 1   서명      708 non-null    object
 2   저자      708 non-null    object
 3   발행자     708 non-null    object
 4   발행년     708 non-null    object
 5   청구기호    708 non-null    object
 6   대출일     708 non-null    object
dtypes: object(7)
memory usage: 44.2+ KB

--- 열 제거 후 데이터프레임 상위 5개 행 (df.head()) ---


,도서관,서명,저자,발행자,발행년,청구기호,대출일
0,화성동탄중앙,매직필 : 기적의 비만 치료제와 살찌지 않는 인간의 탄생,요한 하리 지음 ; 이지연 옮김,어크로스,2025,513.398-하239ㅁ,2025/11/17
1,봉담,복지의 원리 : 대한민국 복지를 한눈에 꿰뚫는 10가지 이야기,양재진 지음,한겨레,2020,338.1-양72ㅂ,2025/11/08
2,화성동탄중앙,모든 것이 전쟁이다: 전 세계를 집어삼킨 아마존의 단 하나의 원칙,다나 마티올리 지음 ; 이영래 옮김,21세기북스,2024,325.1-마839ㅁ,2025/10/27
3,화성동탄중앙,"마음의 여섯 얼굴 : 우울, 불안, 분노, 중독, 광기, 그리고 사랑에 대하여",김건종 지음,에이도스,2019,181.7-김13ㅁ,2025/10/27
4,병점,기울어진 평등 : 부와 권력은 왜 불평등을 허락하는가,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김",와이즈베리,2025,332.6-피823ㄱ,2025/10/04


# '저자'열 다듬기

In [24]:
AUTHOR_COLUMN = '저자'

# 1. 모든 저자 문자열을 하나의 긴 문자열로 합칩니다.
all_author_text = ' '.join(df[AUTHOR_COLUMN].astype(str).fillna('').tolist())

# 2. 한글, 영어 알파벳, 숫자, 공백을 제외한 모든 문자열을 추출합니다.
pattern_to_extract_pure_special = r'([^\s\w가-힣])'

# 3. 모든 일치 항목을 찾습니다. (단일 문자 단위)
all_tokens = re.findall(pattern_to_extract_pure_special, all_author_text)

# 4. 추출된 특수 문자들의 빈도를 계산합니다.
token_counts = Counter(all_tokens)
sorted_counts = sorted(token_counts.items(), key=lambda item: (-item[1], item[0]))

# 5. 분석 결과를 저장할 리스트 초기화
analysis_results = []
ALL_SPECIAL_CHARS = [] 

print("\n--- 저자 컬럼 순수 특수 문자 분석 시작 ---")

for token, count in sorted_counts:
    # 6-1. 특수 문자 이스케이프 처리
    escaped_token = re.escape(token)
    
    # 6-2. 해당 특수 문자를 포함하는 행만 필터링합니다.
    rows_containing_token = df[AUTHOR_COLUMN].astype(str).str.contains(escaped_token, regex=True, na=False)
    
    # 6-3. 필터링된 행의 개수를 계산합니다.
    row_count = rows_containing_token.sum()
    
    # 6-4. 결과를 리스트에 딕셔너리 형태로 저장합니다.
    analysis_results.append({
        "특수 문자": token,
        "총 횟수": count,
        "포함 행 개수": row_count
    })
    
    # 6-5. 현재 특수 문자를 최종 리스트에 추가합니다.
    ALL_SPECIAL_CHARS.append(token)

# 7. 분석 결과를 데이터프레임으로 변환하여 표로 출력
if analysis_results:
    result_df = pd.DataFrame(analysis_results)
    print("--- 특수 문자 분석 결과 표 ---")
    
    # 📌 핵심: 변수명만 단독으로 두어 깔끔한 HTML 표로 렌더링
    display(result_df) 
else:
    print("특수 문자가 발견되지 않았습니다.")


# 8. 최종 특수 문자 리스트 확인 (이 부분은 print로 출력합니다)
print("\n" + "=" * 50)
print("✅ 최종 특수 문자 리스트 (ALL_SPECIAL_CHARS):")
print(ALL_SPECIAL_CHARS)
print("=" * 50)


--- 저자 컬럼 순수 특수 문자 분석 시작 ---
--- 특수 문자 분석 결과 표 ---


,특수 문자,총 횟수,포함 행 개수
0,;,556,447
1,[,123,115
2,],123,115
3,",",80,60
4,·,65,65
5,.,58,44
6,:,6,4
7,(,2,2
8,),2,2
9,/,2,2



✅ 최종 특수 문자 리스트 (ALL_SPECIAL_CHARS):
[';', '[', ']', ',', '·', '.', ':', '(', ')', '/', '-']


In [25]:
AUTHOR_COLUMN = '저자'

# 'ALL_SPECIAL_CHARS' 리스트가 이전 셀에서 성공적으로 생성되었다고 가정합니다.

# 개별 특수 문자별로 반복하며 필터링 및 샘플 확인
for char in ALL_SPECIAL_CHARS:
    
    # 1. 특수 문자 이스케이프 처리
    escaped_char = re.escape(char)
    
    # 2. str.contains()를 사용하여 해당 특수 문자가 포함된 행만 필터링합니다.
    df_char_filtered = df[df[AUTHOR_COLUMN].astype(str).str.contains(escaped_char, regex=True, na=False)]
    
    count = len(df_char_filtered)
    
    # 구분선 및 텍스트 출력
    print("=" * 70)
    print(f"[{char}] 특수 문자가 포함된 행: {count}개")
    print("=" * 70)
    
    if count > 0:
        print(f"--- '{char}' 포함 저자 샘플 (최대 20개) ---")
        
        # 3. 필터링된 데이터프레임의 '저자' 컬럼 상위 20개 샘플을 추출합니다.
        sample_df = df_char_filtered[[AUTHOR_COLUMN]].head(20).reset_index(drop=True)
        
        # 📌 핵심: display()를 사용하여 데이터프레임을 HTML 표로 출력합니다.
        display(sample_df)
        
        print("\n")
    else:
        print("포함된 행이 없습니다.\n")

[;] 특수 문자가 포함된 행: 447개
--- ';' 포함 저자 샘플 (최대 20개) ---


,저자
0,요한 하리 지음 ; 이지연 옮김
1,다나 마티올리 지음 ; 이영래 옮김
2,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김"
3,"데이비드 켈리, 톰 켈리 [공]지음 ; MX디자인랩 옮김"
4,"리처드 도킨스 [외]지음 ; 존 브록만, 카딘카 매트슨[공]엮음, 김동광 옮김"
5,"데이비드 로버트슨, 빌 브린 지음 ; 김태훈 옮김"
6,올리버 프랭클린-월리스 지음 ; 김문주 옮김
7,"잭 트라우트, 앨 리스 [공]지음 ; 안진환 옮김"
8,숀 스티븐슨 지음 ; 최명희 옮김
9,이나모리 가즈오 지음 ; 김윤경 옮김




[[] 특수 문자가 포함된 행: 115개
--- '[' 포함 저자 샘플 (최대 20개) ---


,저자
0,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김"
1,"전홍재, 최성훈, 권창일 [공]지음"
2,"김용태, 류지곤, 이상협 [공]저"
3,"데이비드 켈리, 톰 켈리 [공]지음 ; MX디자인랩 옮김"
4,"리처드 도킨스 [외]지음 ; 존 브록만, 카딘카 매트슨[공]엮음, 김동광 옮김"
5,"잭 트라우트, 앨 리스 [공]지음 ; 안진환 옮김"
6,"후지이 야스후미, 코시로 타카시, 사토 슌 [공]지음 ; 김영혜 옮김"
7,나홍석 [외] 지음
8,"이준구, 최승주 [공]지음"
9,"칩 히스, 댄 히스 [공]지음 ; 안진환, 박슬라 [공]옮김"




[]] 특수 문자가 포함된 행: 115개
--- ']' 포함 저자 샘플 (최대 20개) ---


...

,저자
0,이성경(바이블) 지음
1,소수몽키(홍승초) 지음




[/] 특수 문자가 포함된 행: 2개
--- '/' 포함 저자 샘플 (최대 20개) ---


,저자
0,나는 왜 KBS에서 해임되었나 / 정연주 지음
1,이동수 글/그림




[-] 특수 문자가 포함된 행: 1개
--- '-' 포함 저자 샘플 (최대 20개) ---


,저자
0,올리버 프랭클린-월리스 지음 ; 김문주 옮김


# 하드코딩으로 제거

In [28]:
# 저자 컬럼 정제 규칙 맵 (Key: 바꿀 문자열, Value: 바뀔 문자열)
CLEANING_MAP = {
    "올리버 프랭클린-월리스 지음 ; 김문주 옮김": "올리버 프랭클린-월리스",
    "나는 왜 KBS에서 해임되었나 / 정연주 지음": "정연주",
    "이동수 글/그림": "이동수",
    "지은이: 김원영": "김원영",
    "지은이: 프란츠 카프카 ; 옮긴이: 이재황": "프란츠 카프카",
    "지은이: 잭 슈웨거 ; 옮긴이: 김인정": "잭 슈웨거",
    "수 클리볼드 ; 옮긴이: 홍한별": "수 클리볼드",
    "박종관 글구성.그림": "박종관",
    "나관중 원작. 김우영 그림": "나관중"
}

print("✅ 초기 정제 규칙 맵이 생성되었습니다.")
print("현재 규칙 맵 (CLEANING_MAP):")
print(CLEANING_MAP)

# 0. 정제 전 저자 데이터 복사 (선택 사항: Before 비교용)
# '저자_원본' 컬럼이 없다면 새로 생성합니다.
if '저자_원본' not in df.columns:
    df['저자_원본'] = df[AUTHOR_COLUMN] 

# 1. 1:1 하드코딩 치환 적용
# .replace(dict)를 사용하여 맵에 있는 원본 문자열을 최종 문자열로 정확히 치환합니다.
df[AUTHOR_COLUMN] = df[AUTHOR_COLUMN].replace(CLEANING_MAP)

print("✅ 저자 컬럼 1:1 하드코딩 치환 (CLEANING_MAP) 적용 완료!")

✅ 초기 정제 규칙 맵이 생성되었습니다.
현재 규칙 맵 (CLEANING_MAP):
{'올리버 프랭클린-월리스 지음 ; 김문주 옮김': '올리버 프랭클린-월리스', '나는 왜 KBS에서 해임되었나 / 정연주 지음': '정연주', '이동수 글/그림': '이동수', '지은이: 김원영': '김원영', '지은이: 프란츠 카프카 ; 옮긴이: 이재황': '프란츠 카프카', '지은이: 잭 슈웨거 ; 옮긴이: 김인정': '잭 슈웨거', '수 클리볼드 ; 옮긴이: 홍한별': '수 클리볼드', '박종관 글구성.그림': '박종관', '나관중 원작. 김우영 그림': '나관중'}
✅ 저자 컬럼 1:1 하드코딩 치환 (CLEANING_MAP) 적용 완료!


# 함수로 제거

### 마침표 제거

In [29]:
def clean_author_by_pattern(author_string):
    """
    1. '글.' 패턴을 찾아 해당 패턴과 오른쪽 모든 문자를 제거하고 
       그 왼쪽에 있는 공백까지 제거합니다.
    2. '...' (점 세 개) 문자를 제거합니다.
    """
    if not isinstance(author_string, str):
        return author_string
        
    # 1. '...' (점 세 개) 제거
    cleaned_string = author_string.replace('...', '')
    
    # 2. '글.' 패턴 제거 (공백 포함 오른쪽 전체 제거)
    # 패턴: [0개 이상의 공백] + "글." + [나머지 모든 문자]
    pattern_dot = r'\s*글\..*$'
    cleaned_string = re.sub(pattern_dot, '', cleaned_string)
    
    return cleaned_string
df[AUTHOR_COLUMN] = df[AUTHOR_COLUMN].apply(clean_author_by_pattern)

# 1. 마침표(.)를 정규 표현식 문자로 이스케이프 처리
# '\.'는 문자 그대로의 마침표를 찾습니다.
escaped_dot = re.escape('.')

# 2. str.contains()를 사용하여 마침표가 포함된 행만 필터링합니다.
df_dot_filtered = df[df[AUTHOR_COLUMN].astype(str).str.contains(escaped_dot, regex=True, na=False)]

count = len(df_dot_filtered)

print("=" * 70)
print(f"[.] (점/마침표) 문자가 포함된 행: {count}개")
print("=" * 70)

if count > 0:
    print(f"--- '.' 포함 저자 샘플 (최대 20개) ---")
    
    # 3. 필터링된 데이터프레임의 '저자' 컬럼 상위 20개 샘플을 추출합니다.
    sample_df = df_dot_filtered[[AUTHOR_COLUMN]].head(50).reset_index(drop=True)
    
    # 📌 핵심: display()를 사용하여 데이터프레임을 HTML 표로 출력합니다.
    display(sample_df)
    
    print("\n")
else:
    print("포함된 행이 없습니다.")

[.] (점/마침표) 문자가 포함된 행: 17개
--- '.' 포함 저자 샘플 (최대 20개) ---


,저자
0,브루스 J. 딕슨 지음 ; 박우 옮김
1,벤 S. 버냉키 지음 ; 김동규 옮김
2,대니얼 J. 레비틴 지음 ; 김성훈 옮김
3,마셜 B. 로젠버그 지음 ; 캐서린 한 옮김
4,"Paul R. Krugman, Maurice Obstfeld, Marc J. Mel..."
5,탈레스 S. 테이셰이라 지음 ; 김인수 옮김
6,Aaron T. Beck 지음 ; 김현수 [외]옮김
7,L. 랜덜 레이 지음 ; 홍기빈 옮김
8,조던 B. 피터슨 지음 ; 강주헌 옮김
9,클레이튼 M. 크리스텐슨 ; 마이클 E. 레이너 [공]지음 ; 딜로이트컨설팅코리아 옮김


### 가운데 점 제거

In [33]:
def clean_author_by_pattern(author_string):
    """
    1. '...' (점 세 개) 문자를 제거합니다.
    2. '·' 패턴을 찾아 해당 단어와 오른쪽 전체 문자를 제거합니다.
    3. '글·그림' 패턴을 찾아 해당 패턴과 오른쪽 모든 문자를 제거합니다.
    4. '글.' 패턴을 찾아 해당 패턴과 오른쪽 모든 문자를 제거합니다.
    """
    if not isinstance(author_string, str):
        return author_string
        
    cleaned_string = author_string
    
    # 1. '...' (점 세 개) 제거
    cleaned_string = cleaned_string.replace('...', '')
    
    # 2. '·' 패턴 제거: '·'을 포함하는 단어와 오른쪽 전체를 제거합니다.
    # 패턴: [0개 이상의 공백] + [띄어쓰기 아닌 문자(단어)] + '·' + [나머지 모든 문자]
    pattern_middle_dot = r'\s*[^ ]+·.*$'
    cleaned_string = re.sub(pattern_middle_dot, '', cleaned_string)
    
    # 3. '글·그림' 패턴 제거 (글·그림 자체와 오른쪽 전체를 제거)
    pattern_middle_dot_specific = r'\s*글·그림.*$'
    cleaned_string = re.sub(pattern_middle_dot_specific, '', cleaned_string)
    
    # 4. '글.' 패턴 제거 (글. 자체와 오른쪽 전체를 제거)
    pattern_dot = r'\s*글\..*$'
    cleaned_string = re.sub(pattern_dot, '', cleaned_string)
    
    return cleaned_string.strip() # 최종적으로 남은 공백 제거

# 저자 컬럼에 함수 적용
df[AUTHOR_COLUMN] = df[AUTHOR_COLUMN].apply(clean_author_by_pattern)

print(f"✅ '·' 포함 단어 제거를 포함한 일괄 패턴 변형 함수가 {AUTHOR_COLUMN} 컬럼에 적용되었습니다.")
print("--- 저자 컬럼 정제 전/후 비교 (최대 20개 샘플) ---")

# '저자_원본'과 '저자' 컬럼만 추출하여 상위 20개 행 출력
# 이전에 정의된 '저자_원본' 컬럼이 정제 전 데이터를 담고 있다고 가정합니다.
comparison_df = df[['저자_원본', AUTHOR_COLUMN]].head(20).reset_index(drop=True)

# 깔끔한 표로 출력
display(comparison_df)


# 저자 컬럼에 함수 적용
df[AUTHOR_COLUMN] = df[AUTHOR_COLUMN].apply(clean_author_by_pattern)

print(f"✅ 일괄 패턴 변형 함수(clean_author_by_pattern)가 {AUTHOR_COLUMN} 컬럼에 적용되었습니다.")

# 1. 가운데 점(·)을 정규 표현식 문자로 이스케이프 처리
escaped_middle_dot = re.escape('·')

# 2. str.contains()를 사용하여 가운데 점이 포함된 행만 필터링합니다.
df_dot_filtered = df[df[AUTHOR_COLUMN].astype(str).str.contains(escaped_middle_dot, regex=True, na=False)]

count = len(df_dot_filtered)

print("=" * 70)
print(f"[·] (가운데 점) 문자가 포함된 행: 총 {count}개")
print("=" * 70)

if count > 0:
    print(f"--- '·' 포함 저자 전체 목록 ---")
    
    # 3. 필터링된 전체 데이터프레임의 '저자' 컬럼만 추출하여 출력합니다.
    # 인덱스를 재설정하여 보기 좋게 만듭니다.
    full_df_to_display = df_dot_filtered[[AUTHOR_COLUMN]].reset_index(drop=True)
    
    # 📌 핵심: display()를 사용하여 전체 데이터프레임을 HTML 표로 출력합니다.
    display(full_df_to_display)
    
else:
    print("포함된 행이 없습니다.")

✅ '·' 포함 단어 제거를 포함한 일괄 패턴 변형 함수가 저자 컬럼에 적용되었습니다.
--- 저자 컬럼 정제 전/후 비교 (최대 20개 샘플) ---


,저자_원본,저자
0,요한 하리 지음 ; 이지연 옮김,요한 하리 지음 ; 이지연 옮김
1,양재진 지음,양재진 지음
2,다나 마티올리 지음 ; 이영래 옮김,다나 마티올리 지음 ; 이영래 옮김
3,김건종 지음,김건종 지음
4,"토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김","토마 피케티, 마이클 샌델 [공]지음 ; 장경덕 옮김"
5,서암 스님 지음,서암 스님 지음
6,서암 스님 지음,서암 스님 지음
7,법정 지음,법정 지음
8,"전홍재, 최성훈, 권창일 [공]지음","전홍재, 최성훈, 권창일 [공]지음"
9,"김용태, 류지곤, 이상협 [공]저","김용태, 류지곤, 이상협 [공]저"


✅ 일괄 패턴 변형 함수(clean_author_by_pattern)가 저자 컬럼에 적용되었습니다.
[·] (가운데 점) 문자가 포함된 행: 총 0개
포함된 행이 없습니다.
